# 🖥️ Building a Multi-Agent RAG System  

[Source Link](https://colab.research.google.com/drive/1bDpdyUE8CHESJ94iYgKws-U903SktfH8?usp=sharing#scrollTo=ef1aba59)

## Introduction  
- In this series of exercises, you'll learn how to:  
    - Implement a multi-agent Retrieval-Augmented Generation (RAG) system.  
    - Make LLM API calls via Together.AI.  
    - Create user interfaces for AI applications using Gradio.  
- We'll integrate Gradio’s interactive UI components with large language models to develop a multi-agent RAG system that enhances information retrieval and generation.  
- Each section includes a Module to test different LLM-based Capabilities

## Submission Instructions  
- Complete the Task 1 at the end of the notebook and share your demo with the group.  



## 👀 Module 0: Define a Function for Prompting LLaMA 🦙

### 📝 What does this cell do?
- This code creates two functions for us that makes it easy for us to interact with Llama.
- Llama is a large language model much like ChatGPT


In [ ]:
from ApiKey import API_KEY, HGToken
# import libraries
import requests
from PIL import Image

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

try:
    import gradio as gr
    from together import Together

except ImportError:
    %pip install -q together
    %pip install -q gradio

import gradio as gr
from together import Together

# Get Client
client = Together(api_key=API_KEY)

def prompt_llm(prompt, show_cost=False):
    # This function allows us to prompt an LLM via the Together API

    # model
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"

    # Calculate the number of tokens
    tokens = len(prompt.split())

    # Calculate and print estimated cost for each model
    if show_cost:
        print(f"\nNumber of tokens: {tokens}")
        cost = (0.1 / 1_000_000) * tokens
        print(f"Estimated cost for {model}: ${cost:.10f}\n")

    # Make the API call
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

def gen_image(prompt, width=256, height=256):
    # This function allows us to generate images from a prompt
    response = client.images.generate(
        prompt=prompt,
        model="stabilityai/stable-diffusion-xl-base-1.0",  # Using a supported model
        steps=30,
        n=1,
    )
    image_url = response.data[0].url
    image_filename = "image.png"

    # Download the image using requests instead of wget
    response = requests.get(image_url)
    with open(image_filename, "wb") as f:
        f.write(response.content)
    img = Image.open(image_filename)
    img = img.resize((height, width))

    return img

print("LLM Ready!")


LLM Ready!


## Module 2: Process Emails with multi agents

- This code defines an EmailAgent class that can analyze emails, draft responses, and review responses based on the assigned role.

- It processes sample emails by first analyzing them for key details, then drafting appropriate responses, and finally reviewing the drafts for quality before displaying the results.

In [2]:
class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client

    def process(self, content):
        prompts = {
            # Analyzer prompt - extracts key information from email
            "analyzer": """SYSTEM: You are an expert email analyzer with years of experience in professional communication. Your role is to break down emails into their key components and provide clear, actionable insights.

            As an email analyzer, examine this email content and extract:
            1. Main topics and key points
            2. Urgency level
            3. Required actions
            4. Tone of the message

            INSTRUCTIONS:
            • Focus on extracting factual information without interpretation
            • Identify any deadlines or time-sensitive elements
            • Categorize the email priority (high/medium/low)
            • Show output only - no explanations or additional commentary

            Email: {content}

            Provide a structured analysis.""",
            # Drafter prompt - creates email response based on analysis
            "drafter": """SYSTEM: You are a professional email response specialist with extensive experience in business communication. Your role is to craft clear, effective, and appropriate email responses based on provided analysis.

            As an email response drafter, using this analysis: {content}
            Create a professional email response that:
            1. Addresses all key points
            2. Matches the appropriate tone
            3. Includes clear next steps

            INSTRUCTIONS:
            • Maintain consistent professional tone throughout response
            • Include specific details from the analysis
            • End with clear actionable next steps
            • Show output only - provide just the email response

            Write the complete response.""",
            # Reviewer prompt - evaluates the draft response
            "reviewer": """SYSTEM: You are a senior email quality assurance specialist with a keen eye for detail and professional standards. Your role is to ensure all email responses meet the highest standards of business communication.

            As an email quality reviewer, evaluate this draft response: {content}
            Check for:
            1. Professionalism and appropriateness
            2. Completeness (all points addressed)
            3. Clarity and tone
            4. Potential improvements

            INSTRUCTIONS:
            • Verify all original questions/requests are addressed
            • Check for appropriate formality and politeness
            • Ensure response is concise and well-structured
            • Show output only - return APPROVED or NEEDS_REVISION with brief feedback

            Return either APPROVED or NEEDS_REVISION with specific feedback.""",
        }

        return prompt_llm(prompts[self.role].format(content=content))


if __name__ == "__main__":
    print("\n\nWelcome to the Email Processing System!\n")
    # Example usage with email content
    sample_emails = [
        """Subject: Project Update Request
        Hi team, I hope this email finds you well. Could you please provide an update
        on the current status of the ML project? We need to know the timeline for
        the next deliverable. Thanks!""",
        """Subject: Meeting Scheduling
        Hello, I'd like to schedule a meeting to discuss the recent developments.
        Would tomorrow at 2 PM work for you? Best regards.""",
    ]

    # Process the sample emails
    analyzer = EmailAgent("analyzer", client)
    drafter = EmailAgent("drafter", client)
    reviewer = EmailAgent("reviewer", client)

    for i, email_content in enumerate(sample_emails):
        print(f"\nProcessing email {i+1}")

        # Step 1: Analyze email
        print("\nAnalyzing email content...")
        analysis = analyzer.process(email_content)

        # Step 2: Draft response
        print("\nDrafting response based on analysis...")
        draft = drafter.process(analysis)

        # Display formatted output
        print("\n" + "=" * 50)
        print("ORIGINAL EMAIL:\n")
        print(email_content)
        print("\n" + "=" * 50)
        print("DRAFT RESPONSE:\n")
        print(draft)
        print("\n" + "=" * 50)
        # input("Press Enter to continue...")




Welcome to the Email Processing System!


Processing email 1

Analyzing email content...

Drafting response based on analysis...

ORIGINAL EMAIL:

Subject: Project Update Request
        Hi team, I hope this email finds you well. Could you please provide an update
        on the current status of the ML project? We need to know the timeline for
        the next deliverable. Thanks!

DRAFT RESPONSE:

Subject: Project Update and Next Deliverable Timeline

Dear [Recipient],

I hope this email finds you well. I am writing to provide a project update and to confirm the timeline for the next deliverable. As per our previous discussions, I am pleased to report that we are making good progress on the project. Our team has been working diligently to ensure that all tasks are completed on schedule, and we are on track to meet the project milestones.

Regarding the next deliverable, I am pleased to confirm that it is scheduled to be completed by [Date]. Our team is working hard to ensure that i

## Module 3: Gradio User Interface for Processing Emails

- This code defines an Email Processing System that can analyze an email, draft a response, and display both results using a web-based interface.

- It uses Gradio to create an interactive webpage where users can input an email, click a button to process it, and see the AI-generated analysis and response.

In [3]:
class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client

    def process(self, content):
        prompts = {
            # Analyzer prompt - extracts key information from email
            "analyzer": """SYSTEM: You are an expert email analyzer with years of experience in professional communication. Your role is to break down emails into their key components and provide clear, actionable insights.

            As an email analyzer, examine this email content and extract:
            1. Main topics and key points
            2. Urgency level
            3. Required actions
            4. Tone of the message

            INSTRUCTIONS:
            • Focus on extracting factual information without interpretation
            • Identify any deadlines or time-sensitive elements
            • Categorize the email priority (high/medium/low)
            • Show output only - no explanations or additional commentary

            Email: {content}

            Provide a structured analysis.""",
            # Drafter prompt - creates email response based on analysis
            "drafter": """SYSTEM: You are a professional email response specialist with extensive experience in business communication. Your role is to craft clear, effective, and appropriate email responses based on provided analysis.

            As an email response drafter, using this analysis: {content}
            Create a professional email response that:
            1. Addresses all key points
            2. Matches the appropriate tone
            3. Includes clear next steps

            INSTRUCTIONS:
            • Maintain consistent professional tone throughout response
            • Include specific details from the analysis
            • End with clear actionable next steps
            • Show output only - provide just the email response

            Write the complete response.""",
            # Reviewer prompt - evaluates the draft response
            "reviewer": """SYSTEM: You are a senior email quality assurance specialist with a keen eye for detail and professional standards. Your role is to ensure all email responses meet the highest standards of business communication.

            As an email quality reviewer, evaluate this draft response: {content}
            Check for:
            1. Professionalism and appropriateness
            2. Completeness (all points addressed)
            3. Clarity and tone
            4. Potential improvements

            INSTRUCTIONS:
            • Verify all original questions/requests are addressed
            • Check for appropriate formality and politeness
            • Ensure response is concise and well-structured
            • Show output only - return APPROVED or NEEDS_REVISION with brief feedback

            Return either APPROVED or NEEDS_REVISION with specific feedback.""",
        }

        return prompt_llm(prompts[self.role].format(content=content))


def process_email(email_content):
    # Create agents
    analyzer = EmailAgent("analyzer", client)
    drafter = EmailAgent("drafter", client)

    # Process email
    analysis = analyzer.process(email_content)
    draft = drafter.process(analysis)

    return analysis, draft


# Example emails
example_emails = [
    """Dear Team,
I hope this email finds you well. We need to reschedule tomorrow's project meeting due to a conflict. Could we move it to Friday at 2 PM instead?
Best regards,
John""",
    """Subject: Urgent: Server Downtime
The production server is currently experiencing issues. We need immediate assistance to resolve this. Please respond ASAP.
-Sarah from DevOps""",
    """Hi Marketing Team,
Just wanted to follow up on the Q4 report. When can we expect the first draft for review?
Thanks,
Mike""",
]


class EmailDemo:
    def __init__(self):
        self.current_index = 0

    def get_current_email(self):
        return example_emails[self.current_index]

    def next_email(self, _):
        self.current_index = (self.current_index + 1) % len(example_emails)
        return example_emails[self.current_index]


demo_state = EmailDemo()

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 📧 Email Processing System")
    gr.Markdown("View example emails and get AI-powered analysis and responses.")

    with gr.Row():
        email_input = gr.Textbox(
            value=demo_state.get_current_email(), lines=5, label="📝 Email Content"
        )
        next_button = gr.Button("⏭️ Next Example Email")

    process_button = gr.Button("🔄 Process Email")

    with gr.Row():
        analysis_output = gr.Textbox(
            lines=8, label="📊 Analysis", show_copy_button=True
        )
        draft_output = gr.Textbox(
            lines=8, label="✉️ Draft Response", show_copy_button=True
        )

    # Set up event handlers
    next_button.click(
        demo_state.next_email, inputs=[email_input], outputs=[email_input]
    )

    process_button.click(
        process_email, inputs=[email_input], outputs=[analysis_output, draft_output]
    )

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


## Module 4: RAG to find the Best Matching Policy

- This code finds the most relevant medical policy based on a user's question by comparing the meaning of their query with stored policies using AI-powered text similarity.

- It processes example questions, retrieves the best-matching policy, and displays its content along with a similarity score to show how well the policy matches the question.

In [27]:
# Import necessary libraries
try:
    import sentence_transformers
except ImportError:
    %pip install -q sentence-transformers
    import sentence_transformers  # Re-import after installation

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from ApiKeyHG import HGToken

class PolicyRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer(
            "all-MiniLM-L6-v2", use_auth_token=HGToken
        )
        # Sample medical policies - in production, this would come from a database
        self.policies = {
            "privacy": """
                Patient Privacy Policy:
                - All patient information is confidential and protected under HIPAA
                - Access to medical records requires patient consent
                - Data sharing with third parties strictly regulated
            """,
            "appointments": """
                Appointment Policy:
                - 24-hour notice required for cancellations
                - Telehealth options available for eligible consultations
                - Emergency cases prioritized based on severity
            """,
            "insurance": """
                Insurance Policy:
                - We accept major insurance providers
                - Pre-authorization required for specific procedures
                - Co-pay due at time of service
            """,
            "medication": """
                Medication Policy:
                - Prescription refills require 48-hour notice
                - Controlled substances have strict monitoring protocols
                - Generic alternatives offered when available
            """,
        }
        # Pre-compute embeddings for policies
        self.policy_embeddings = {
            k: self.encoder.encode(v) for k, v in self.policies.items()
        }

    def get_relevant_policy(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.policy_embeddings.items()
        }
        sorted_policies = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_policies = []
        for policy_name, score in sorted_policies[:top_k]:
            if score > 0.3:  # Similarity threshold
                relevant_policies.append(self.policies[policy_name])

        return (
            "\n\n".join(relevant_policies)
            if relevant_policies
            else "No relevant policy found."
        )


if __name__ == "__main__":
    # Example usage
    retriever = PolicyRetriever()

    # Test queries with expected policy matches
    test_queries = [
        "I need to cancel my appointment tomorrow morning",
        "Do you share my medical information with other doctors?",
        "When do I need to pay my insurance copay?",
        "How can I get my prescription refilled?",
    ]

    for query in test_queries:
        # Get similarity scores for all policies
        query_embedding = retriever.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in retriever.policy_embeddings.items()
        }

        # Get the most relevant policy and its score
        best_match = max(similarities.items(), key=lambda x: x[1])
        policy_name, score = best_match

        print(f"\nQuery: {query}")
        print(f"Best matching policy: {policy_name}")
        print(f"Similarity score: {score:.3f}")
        print(f"Policy content:\n{retriever.policies[policy_name]}")
        print("-" * 80)



Query: I need to cancel my appointment tomorrow morning
Best matching policy: appointments
Similarity score: 0.550
Policy content:

                Appointment Policy:
                - 24-hour notice required for cancellations
                - Telehealth options available for eligible consultations
                - Emergency cases prioritized based on severity
            
--------------------------------------------------------------------------------

Query: Do you share my medical information with other doctors?
Best matching policy: privacy
Similarity score: 0.584
Policy content:

                Patient Privacy Policy:
                - All patient information is confidential and protected under HIPAA
                - Access to medical records requires patient consent
                - Data sharing with third parties strictly regulated
            
--------------------------------------------------------------------------------

Query: When do I need to pay my insurance copay

## Module 5: RAG that Finds the Best Email Response

- This program finds the most relevant pre-written email response based on a user's question.

- It uses a language model to compare the question with stored email examples and returns the best match.

In [29]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


class EmailResponseRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2", use_auth_token=HGToken)
        # Sample email responses - in production, this would come from a database
        self.examples = {
            "late_delivery": """
                ORIGINAL EMAIL:
                I haven't received my order yet and it's been 2 weeks. This is unacceptable.

                MY RESPONSE:
                I sincerely apologize for the delay in your order. I understand your frustration.
                I've checked with our shipping department and your package is currently in transit.
                I'll expedite this and send you the updated tracking information within the next hour.
                Please let me know if you need anything else.
            """,
            "refund_request": """
                ORIGINAL EMAIL:
                The product I received is damaged. I want my money back immediately.

                MY RESPONSE:
                I'm very sorry to hear about the damaged product. I completely understand your concern.
                I've initiated an immediate refund which will be processed within 2-3 business days.
                Would you like a return shipping label to send the damaged item back to us?
            """,
            "product_inquiry": """
                ORIGINAL EMAIL:
                Does this come in different sizes? And what colors are available?

                MY RESPONSE:
                Thank you for your interest! Yes, this product comes in S, M, L, and XL.
                We currently have it available in navy blue, forest green, and charcoal gray.
                I can provide detailed measurements for any specific size you're interested in.
            """,
            "technical_support": """
                ORIGINAL EMAIL:
                The software keeps crashing when I try to export my project.

                MY RESPONSE:
                I understand how frustrating technical issues can be. Let's resolve this together.
                First, please try clearing your cache and restarting the application.
                If that doesn't work, could you send me your error log? You can find it at Settings > Help > Export Log.
            """,
        }
        # Pre-compute embeddings for examples
        self.example_embeddings = {
            k: self.encoder.encode(v) for k, v in self.examples.items()
        }

    def get_relevant_example(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.example_embeddings.items()
        }
        sorted_examples = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_examples = []
        for example_name, score in sorted_examples[:top_k]:
            if score > 0.3:  # Similarity threshold
                relevant_examples.append(self.examples[example_name])

        return (
            "\n\n".join(relevant_examples)
            if relevant_examples
            else "No relevant example found."
        )


if __name__ == "__main__":
    # Example usage
    retriever = EmailResponseRetriever()

    # Test queries with expected example matches
    test_queries = [
        "My package hasn't arrived yet",
        "I got a broken item in the mail",
        "What sizes do you have?",
        "The app keeps crashing",
    ]

    for query in test_queries:
        # Get similarity scores for all examples
        query_embedding = retriever.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in retriever.example_embeddings.items()
        }

        # Get the most relevant example and its score
        best_match = max(similarities.items(), key=lambda x: x[1])
        example_name, score = best_match

        print(f"\nQuery: {query}")
        print("-" * 80)
        print(f"Best matching example: {example_name}")
        print(f"Similarity score: {score:.3f}")
        print(f"Example content:\n{retriever.examples[example_name]}")
        print("-" * 80)



Query: My package hasn't arrived yet
--------------------------------------------------------------------------------
Best matching example: late_delivery
Similarity score: 0.642
Example content:

                ORIGINAL EMAIL:
                I haven't received my order yet and it's been 2 weeks. This is unacceptable.

                MY RESPONSE:
                I sincerely apologize for the delay in your order. I understand your frustration.
                I've checked with our shipping department and your package is currently in transit.
                I'll expedite this and send you the updated tracking information within the next hour.
                Please let me know if you need anything else.
            
--------------------------------------------------------------------------------

Query: I got a broken item in the mail
--------------------------------------------------------------------------------
Best matching example: refund_request
Similarity score: 0.619
Example

## Module 6: AI-Powered Email Processing for Medical Policies
- This program analyzes, drafts, and reviews email responses by retrieving relevant medical policies and ensuring compliance.

- It uses AI models to process emails, determine sentiment, suggest policy-based responses, and refine drafts for clarity and adherence to regulations.


In [31]:
# Add imports for RAG functionality
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class PolicyRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2", use_auth_token=HGToken)
        # Sample medical policies - in production, this would come from a database
        self.policies = {
            "privacy": """
                Patient Privacy Policy:
                - All patient information is confidential and protected under HIPAA
                - Access to medical records requires patient consent
                - Data sharing with third parties strictly regulated
            """,
            "appointments": """
                Appointment Policy:
                - 24-hour notice required for cancellations
                - Telehealth options available for eligible consultations
                - Emergency cases prioritized based on severity
            """,
            "insurance": """
                Insurance Policy:
                - We accept major insurance providers
                - Pre-authorization required for specific procedures
                - Co-pay due at time of service
            """,
            "medication": """
                Medication Policy:
                - Prescription refills require 48-hour notice
                - Controlled substances have strict monitoring protocols
                - Generic alternatives offered when available
            """,
        }
        # Pre-compute embeddings for policies
        self.policy_embeddings = {
            k: self.encoder.encode(v) for k, v in self.policies.items()
        }

    def get_relevant_policy(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.policy_embeddings.items()
        }
        sorted_policies = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_policies = []
        for policy_name, score in sorted_policies[:top_k]:
            if score > 0.3:  # Similarity threshold
                relevant_policies.append(self.policies[policy_name])

        return (
            "\n\n".join(relevant_policies)
            if relevant_policies
            else "No relevant policy found."
        )


def prompt_llm(prompt, show_cost=False):
    # This function allows us to prompt an LLM via the Together API

    # model
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"

    # Calculate the number of tokens
    tokens = len(prompt.split())

    # Calculate and print estimated cost for each model
    if show_cost:
        print(f"\nNumber of tokens: {tokens}")
        cost = (0.1 / 1_000_000) * tokens
        print(f"Estimated cost for {model}: ${cost:.10f}\n")

    # Make the API call
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


def gen_image(prompt, width=256, height=256):
    # This function allows us to generate images from a prompt
    response = client.images.generate(
        prompt=prompt,
        model="stabilityai/stable-diffusion-xl-base-1.0",  # Using a supported model
        steps=30,
        n=1,
    )
    image_url = response.data[0].url
    image_filename = "image.png"

    # Download the image using requests instead of wget
    response = requests.get(image_url)
    with open(image_filename, "wb") as f:
        f.write(response.content)
    img = Image.open(image_filename)
    img = img.resize((height, width))

    return img


class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client
        self.policy_retriever = PolicyRetriever()

        # Move prompts to be an instance variable
        self.prompts = {
            "analyzer": """SYSTEM: You are an expert email analyzer for a medical company.
            Your role is to break down emails into key components and provide clear, actionable insights.

            INSTRUCTIONS:
            • Extract main topics and key points from the email
            • Determine urgency level (Low, Medium, High)
            • List all required actions in bullet points
            • Analyze tone of the message (formal, informal, urgent, etc.)
            • Identify relevant company policies that apply
            • Highlight any compliance concerns
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            CONTEXT (Company Policies):
            {policies}

            Email: {content}""",
            "drafter": """SYSTEM: You are a professional email response specialist for a medical company.
            Draft responses that align with our policies and maintain HIPAA compliance.

            INSTRUCTIONS:
            • Address all key points from the original email
            • Ensure response aligns with provided company policies
            • Verify HIPAA compliance in all content
            • Include clear next steps and action items
            • Maintain professional and empathetic tone
            • Add necessary disclaimers where applicable
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            CONTEXT (Relevant Policies):
            {policies}

            Based on this analysis: {content}""",
            "reviewer": """SYSTEM: You are a senior email quality assurance specialist for a medical company.
            Ensure responses meet healthcare communication standards and comply with policies.

            INSTRUCTIONS:
            • Verify compliance with all relevant policies
            • Check for HIPAA violations
            • Assess professional tone and clarity
            • Review completeness of response
            • Evaluate appropriate handling of sensitive information
            • Confirm all action items are clearly stated
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            CONTEXT (Relevant Policies):
            {policies}

            Evaluate this draft response: {content}""",
            "sentiment": """SYSTEM: You are an expert in analyzing email sentiment and emotional context in
            healthcare communications.

            INSTRUCTIONS:
            • Analyze overall sentiment (positive, negative, neutral)
            • Identify emotional undertones
            • Detect urgency or stress indicators
            • Assess patient/sender satisfaction level
            • Flag any concerning language
            • Recommend tone adjustments if needed
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            Email: {content}""",
            "policy_justifier": """SYSTEM: You are a policy expert. In 2 lines, explain why the following policies are relevant
            to this email content. Be specific and concise.

            Email content: {content}
            Selected policies: {policies}""",
        }

    def process(self, content):
        # Get relevant policies for the email content
        relevant_policies = self.policy_retriever.get_relevant_policy(content)
        return prompt_llm(
            self.prompts[self.role].format(content=content, policies=relevant_policies)
        )


class EmailProcessingSystem:
    def __init__(self, client):
        self.analyzer = EmailAgent("analyzer", client)
        self.drafter = EmailAgent("drafter", client)
        self.reviewer = EmailAgent("reviewer", client)
        self.policy_justifier = EmailAgent("policy_justifier", client)

    def process_email(self, email_content):
        max_attempts = 3
        attempt = 1

        while attempt <= max_attempts:
            print(f"\nProcessing email - Attempt {attempt}")

            # Step 1: Analyze email
            print("\nAnalyzing email content...")
            analysis = self.analyzer.process(email_content)

            # Step 2: Analyze sentiment
            print("\nAnalyzing sentiment...")
            sentiment = prompt_llm(
                self.analyzer.prompts["sentiment"].format(content=email_content)
            )

            # Step 3: Draft response
            print("\nDrafting response based on analysis...")
            draft = self.drafter.process(analysis)

            # Get relevant policies for display
            relevant_policies = self.analyzer.policy_retriever.get_relevant_policy(
                email_content
            )

            # Add policy justification
            policy_justification = self.policy_justifier.process(
                f"Email: {email_content}\nPolicies: {relevant_policies}"
            )

            # Display formatted output
            print("\n" + "=" * 50)
            print("ORIGINAL EMAIL:\n")
            print(email_content)
            print("\n" + "=" * 50)
            print("DRAFT RESPONSE:\n")
            print(draft)
            print("\n" + "=" * 50)
            print("POLICY USED:\n")
            print(relevant_policies)
            print("\nPOLICY JUSTIFICATION:\n")
            print(policy_justification)
            print("\n" + "=" * 50)

            # Ask user for feedback on the draft
            print("\nAre you satisfied with this draft? (y/n)")
            user_feedback = input().lower()

            if user_feedback != "y":
                print("\nMoving to next attempt...")
                attempt += 1
                continue

            # Step 4: Review response
            print("\nReviewing draft response...")
            review = self.reviewer.process(draft)
            print("\nReview completed. Feedback:")
            print(review)

            if "APPROVED" in review:
                return {
                    "status": "success",
                    "analysis": analysis,
                    "final_draft": draft,
                    "review": review,
                }
            else:
                print(f"\nRevision needed. Feedback: {review}")
                attempt += 1

        return {"status": "failed", "message": "Maximum revision attempts reached"}


def process_emails(emails_list):
    email_system = EmailProcessingSystem(client)
    results = {}

    for email in emails_list:
        print(f"\nProcessing email: {email}")
        print("\nWould you like to process this email? (y/n)")

        user_input = input().lower()
        if user_input != "y":
            print("Skipping this email...")
            continue

        result = email_system.process_email(email)
        results[email] = result

    return results


if __name__ == "__main__":
    print("\n\nWelcome to the Medical Company Email Processing System!\n")
    # Updated example with medical context
    sample_emails = [
        """Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.""",
        """Subject: Insurance Coverage Question
        Hi, I'm scheduled for a procedure next week and wanted to confirm if my insurance
        is accepted at your facility. I have BlueCross BlueShield. Best regards.""",
    ]

    results = process_emails(sample_emails)




Welcome to the Medical Company Email Processing System!


Processing email: Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.

Would you like to process this email? (y/n)

Processing email - Attempt 1

Analyzing email content...

Analyzing sentiment...

Drafting response based on analysis...

ORIGINAL EMAIL:

Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.

DRAFT RESPONSE:

Here is a draft response that aligns with the provided company policies and maintains HIPAA compliance:

"Dear [Referring Physician],

Thank you for your request. To access your patient's recent test results, please provide patient consent and confirm your identity. You can access the online portal or contact our office f


## Module 7: AI Email Responder with Past Examples
- This program retrieves and adapts past email responses to draft new replies while ensuring compliance with healthcare policies.
- It analyzes the email, finds similar past responses, drafts a reply in a friendly tone, and reviews it for accuracy and compliance.

In [32]:
# Add imports for RAG functionality
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class EmailResponseRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2", use_auth_token=HGToken)
        # Sample email response examples with more casual tone
        self.examples = {
            "medical_records": """
                ORIGINAL EMAIL:
                Hey there, I was hoping to get my medical records. What do I need to do?

                MY RESPONSE:
                Hi! Happy to help you get those records. We just need a few quick things:
                1. Your signed OK (we'll send you the form)
                2. A quick form to fill out
                3. Your ID
                Just upload everything to our secure portal and we'll take care of the rest! Let me know if you need help.
            """,
            "insurance_verification": """
                ORIGINAL EMAIL:
                Quick question - do you guys take Aetna insurance?

                MY RESPONSE:
                Hey there! Yes, we work with Aetna and most other major insurance companies.
                Could you shoot me your:
                - Member ID
                - Group number
                I'll double-check everything and get back to you super quick (usually within a day).
                Sound good?
            """,
            "appointment_scheduling": """
                ORIGINAL EMAIL:
                Something came up and I need to move my appointment. Help!

                MY RESPONSE:
                Hey! No worries at all - life happens! 😊
                I've got a couple of spots open:
                - Tuesday @ 2pm
                - Wednesday morning at 10am
                Just let me know what works better for you and I'll get it switched right away!
            """,
            "medication_refill": """
                ORIGINAL EMAIL:
                Running low on my meds - need a refill asap!

                MY RESPONSE:
                Hey! Thanks for the heads up about your meds. I'm on it!
                Here's what's happening next:
                1. We'll review your refill request today
                2. Give your pharmacy a call
                3. They should have it ready in 1-2 days
                Need it sooner? Just let me know!
            """,
        }
        # Pre-compute embeddings for examples
        self.example_embeddings = {
            k: self.encoder.encode(v) for k, v in self.examples.items()
        }

    def get_relevant_response(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.example_embeddings.items()
        }
        sorted_examples = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_examples = []
        for example_name, score in sorted_examples[:top_k]:
            if score > 0.3:  # Similarity threshold
                relevant_examples.append(self.examples[example_name])

        return (
            "\n\n".join(relevant_examples)
            if relevant_examples
            else "No relevant example found."
        )


def prompt_llm(prompt, show_cost=False):
    # This function allows us to prompt an LLM via the Together API

    # model
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"

    # Calculate the number of tokens
    tokens = len(prompt.split())

    # Calculate and print estimated cost for each model
    if show_cost:
        print(f"\nNumber of tokens: {tokens}")
        cost = (0.1 / 1_000_000) * tokens
        print(f"Estimated cost for {model}: ${cost:.10f}\n")

    # Make the API call
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


def gen_image(prompt, width=256, height=256):
    # This function allows us to generate images from a prompt
    response = client.images.generate(
        prompt=prompt,
        model="stabilityai/stable-diffusion-xl-base-1.0",  # Using a supported model
        steps=30,
        n=1,
    )
    image_url = response.data[0].url
    image_filename = "image.png"

    # Download the image using requests instead of wget
    response = requests.get(image_url)
    with open(image_filename, "wb") as f:
        f.write(response.content)
    img = Image.open(image_filename)
    img = img.resize((height, width))

    return img


class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client
        self.response_retriever = EmailResponseRetriever()

        self.prompts = {
            "analyzer": """SYSTEM: You are an expert email analyzer for a medical company.
            Your role is to break down emails into key components and provide clear, actionable insights.

            INSTRUCTIONS:
            • Extract main topics and key points from the email
            • Determine urgency level (Low, Medium, High)
            • List all required actions in bullet points
            • Analyze tone of the message (formal, informal, urgent, etc.)
            • Consider similar past responses
            • Highlight any compliance concerns
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Email: {content}""",
            "drafter": """SYSTEM: You are a professional email response specialist for a medical company.
            Draft responses that align with our past successful responses while maintaining HIPAA compliance.

            INSTRUCTIONS:
            • Address all key points from the original email
            • Use a friendly, conversational tone like in our examples
            • Ensure HIPAA compliance in all content
            • Include clear next steps and action items
            • Maintain professional yet approachable tone
            • Add necessary disclaimers where applicable
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Based on this analysis: {content}""",
            "reviewer": """SYSTEM: You are a senior email quality assurance specialist for a medical company.
            Ensure responses meet healthcare communication standards and match our friendly tone.

            INSTRUCTIONS:
            • Verify alignment with example responses
            • Check for HIPAA violations
            • Assess professional yet friendly tone
            • Review completeness of response
            • Evaluate appropriate handling of sensitive information
            • Confirm all action items are clearly stated
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Evaluate this draft response: {content}""",
            "sentiment": """SYSTEM: You are an expert in analyzing email sentiment and emotional context in
            healthcare communications.

            INSTRUCTIONS:
            • Analyze overall sentiment (positive, negative, neutral)
            • Identify emotional undertones
            • Detect urgency or stress indicators
            • Assess patient/sender satisfaction level
            • Flag any concerning language
            • Recommend tone adjustments if needed
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            Email: {content}""",
            "example_justifier": """SYSTEM: You are an example matching expert. In 2 lines, explain why the following example responses are relevant
            to this email content. Be specific and concise.

            Email content: {content}
            Selected examples: {examples}""",
        }

    def process(self, content):
        # Get relevant example responses for the email content
        relevant_examples = self.response_retriever.get_relevant_response(content)
        return prompt_llm(
            self.prompts[self.role].format(content=content, examples=relevant_examples)
        )


class EmailProcessingSystem:
    def __init__(self, client):
        self.analyzer = EmailAgent("analyzer", client)
        self.drafter = EmailAgent("drafter", client)
        self.reviewer = EmailAgent("reviewer", client)
        self.example_justifier = EmailAgent("example_justifier", client)

    def process_email(self, email_content):
        max_attempts = 3
        attempt = 1

        while attempt <= max_attempts:
            print(f"\nProcessing email - Attempt {attempt}")

            # Step 1: Analyze email
            print("\nAnalyzing email content...")
            analysis = self.analyzer.process(email_content)

            # Step 2: Analyze sentiment
            print("\nAnalyzing sentiment...")
            sentiment = prompt_llm(
                self.analyzer.prompts["sentiment"].format(content=email_content)
            )

            # Step 3: Draft response
            print("\nDrafting response based on analysis...")
            draft = self.drafter.process(analysis)

            # Get relevant example responses for display
            relevant_examples = self.analyzer.response_retriever.get_relevant_response(
                email_content
            )

            # Add example justification
            example_justification = self.example_justifier.process(
                f"Email: {email_content}\nExamples: {relevant_examples}"
            )

            # Display formatted output
            print("\n" + "=" * 50)
            print("ORIGINAL EMAIL:\n")
            print(email_content)
            print("\n" + "=" * 50)
            print("DRAFT RESPONSE:\n")
            print(draft)
            print("\n" + "=" * 50)
            print("EXAMPLE USED:\n")
            print(relevant_examples)
            print("\nEXAMPLE JUSTIFICATION:\n")
            print(example_justification)
            print("\n" + "=" * 50)

            # Ask user for feedback on the draft
            print("\nAre you satisfied with this draft? (y/n)")
            user_feedback = input().lower()

            if user_feedback != "y":
                print("\nMoving to next attempt...")
                attempt += 1
                continue

            # Step 4: Review response
            print("\nReviewing draft response...")
            review = self.reviewer.process(draft)
            print("\nReview completed. Feedback:")
            print(review)

            if "APPROVED" in review:
                return {
                    "status": "success",
                    "analysis": analysis,
                    "final_draft": draft,
                    "review": review,
                }
            else:
                print(f"\nRevision needed. Feedback: {review}")
                attempt += 1

        return {"status": "failed", "message": "Maximum revision attempts reached"}


def process_emails(emails_list):
    email_system = EmailProcessingSystem(client)
    results = {}

    for email in emails_list:
        print(f"\nProcessing email: {email}")
        print("\nWould you like to process this email? (y/n)")

        user_input = input().lower()
        if user_input != "y":
            print("Skipping this email...")
            continue

        result = email_system.process_email(email)
        results[email] = result

    return results


if __name__ == "__main__":
    print("\n\nWelcome to the Medical Company Email Processing System!\n")
    # Updated example with medical context
    sample_emails = [
        """Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.""",
        """Subject: Insurance Coverage Question
        Hi, I'm scheduled for a procedure next week and wanted to confirm if my insurance
        is accepted at your facility. I have BlueCross BlueShield. Best regards.""",
    ]

    results = process_emails(sample_emails)




Welcome to the Medical Company Email Processing System!


Processing email: Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.

Would you like to process this email? (y/n)

Processing email - Attempt 1

Analyzing email content...

Analyzing sentiment...

Drafting response based on analysis...

ORIGINAL EMAIL:

Subject: Patient Data Access Request
        Hello, I'm a referring physician and need access to my patient's recent test results.
        What's the procedure for requesting these records? Thanks.

DRAFT RESPONSE:

Here is a draft response that aligns with the given instructions:

**Response:**

Hi! Thank you for reaching out. To request your patient's recent test results, please follow these steps: [insert procedure]. We'll ensure secure transmission of the records. Let me know if you need assistance.

EXAMPLE USED:


             

## Module 7: AI Email Processing with Gradio User Interface
- This program processes medical-related emails, retrieves similar past responses, and generates draft replies while ensuring compliance.
- It includes a Gradio-based interface for reviewing, approving, or disapproving AI-generated email responses interactively.



In [33]:
# Add imports for RAG functionality
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class EmailResponseRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2", use_auth_token=HGToken)
        # Sample email response examples with more casual tone
        self.examples = {
            "medical_records": """
                ORIGINAL EMAIL:
                Hey there, I was hoping to get my medical records. What do I need to do?

                MY RESPONSE:
                Hi! Happy to help you get those records. We just need a few quick things:
                1. Your signed OK (we'll send you the form)
                2. A quick form to fill out
                3. Your ID
                Just upload everything to our secure portal and we'll take care of the rest! Let me know if you need help.
            """,
            "insurance_verification": """
                ORIGINAL EMAIL:
                Quick question - do you guys take Aetna insurance?

                MY RESPONSE:
                Hey there! Yes, we work with Aetna and most other major insurance companies.
                Could you shoot me your:
                - Member ID
                - Group number
                I'll double-check everything and get back to you super quick (usually within a day).
                Sound good?
            """,
            "appointment_scheduling": """
                ORIGINAL EMAIL:
                Something came up and I need to move my appointment. Help!

                MY RESPONSE:
                Hey! No worries at all - life happens! 😊
                I've got a couple of spots open:
                - Tuesday @ 2pm
                - Wednesday morning at 10am
                Just let me know what works better for you and I'll get it switched right away!
            """,
            "medication_refill": """
                ORIGINAL EMAIL:
                Running low on my meds - need a refill asap!

                MY RESPONSE:
                Hey! Thanks for the heads up about your meds. I'm on it!
                Here's what's happening next:
                1. We'll review your refill request today
                2. Give your pharmacy a call
                3. They should have it ready in 1-2 days
                Need it sooner? Just let me know!
            """,
        }
        # Pre-compute embeddings for examples
        self.example_embeddings = {
            k: self.encoder.encode(v) for k, v in self.examples.items()
        }

    def get_relevant_response(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.example_embeddings.items()
        }
        sorted_examples = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_examples = []
        for example_name, score in sorted_examples[:top_k]:
            if score > 0.3:  # Similarity threshold
                relevant_examples.append(self.examples[example_name])

        return (
            "\n\n".join(relevant_examples)
            if relevant_examples
            else "No relevant example found."
        )

class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client
        self.response_retriever = EmailResponseRetriever()

        self.prompts = {
            "analyzer": """SYSTEM: You are an expert email analyzer for a medical company.
            Your role is to break down emails into key components and provide clear, actionable insights.

            INSTRUCTIONS:
            • Extract main topics and key points from the email
            • Determine urgency level (Low, Medium, High)
            • List all required actions in bullet points
            • Analyze tone of the message (formal, informal, urgent, etc.)
            • Consider similar past responses
            • Highlight any compliance concerns
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Email: {content}""",
            "drafter": """SYSTEM: You are a professional email response specialist for a medical company.
            Draft responses that align with our past successful responses while maintaining HIPAA compliance.

            INSTRUCTIONS:
            • Address all key points from the original email
            • Use a friendly, conversational tone like in our examples
            • Ensure HIPAA compliance in all content
            • Include clear next steps and action items
            • Maintain professional yet approachable tone
            • Add necessary disclaimers where applicable
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Based on this analysis: {content}""",
            "reviewer": """SYSTEM: You are a senior email quality assurance specialist for a medical company.
            Ensure responses meet healthcare communication standards and match our friendly tone.

            INSTRUCTIONS:
            • Verify alignment with example responses
            • Check for HIPAA violations
            • Assess professional yet friendly tone
            • Review completeness of response
            • Evaluate appropriate handling of sensitive information
            • Confirm all action items are clearly stated
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            SIMILAR PAST RESPONSES:
            {examples}

            Evaluate this draft response: {content}""",
            "sentiment": """SYSTEM: You are an expert in analyzing email sentiment and emotional context in
            healthcare communications.

            INSTRUCTIONS:
            • Analyze overall sentiment (positive, negative, neutral)
            • Identify emotional undertones
            • Detect urgency or stress indicators
            • Assess patient/sender satisfaction level
            • Flag any concerning language
            • Recommend tone adjustments if needed
            • Limit response to 50 words maximum
            • Show response only without additional commentary

            Email: {content}""",
            "example_justifier": """SYSTEM: You are an example matching expert. In 2 lines, explain why the following example responses are relevant
            to this email content. Be specific and concise.

            Email content: {content}
            Selected examples: {examples}""",
        }

    def process(self, content):
        # Get relevant example responses for the email content
        relevant_examples = self.response_retriever.get_relevant_response(content)
        return prompt_llm(
            self.prompts[self.role].format(content=content, examples=relevant_examples)
        )


class EmailProcessingSystem:
    def __init__(self, client):
        self.drafter = EmailAgent("drafter", client)
        self.reviewer = EmailAgent("reviewer", client)
        self.example_justifier = EmailAgent("example_justifier", client)

    def process_email(self, email_content):
        # Step 1: Analyze sentiment
        sentiment = prompt_llm(
            self.drafter.prompts["sentiment"].format(content=email_content)
        )

        # Step 2: Draft response
        draft = self.drafter.process(email_content)

        # Get relevant example responses for display
        relevant_examples = self.drafter.response_retriever.get_relevant_response(
            email_content
        )

        # Add example justification
        example_justification = self.example_justifier.process(
            f"Email: {email_content}\nExamples: {relevant_examples}"
        )

        # Step 3: Review response
        review = self.reviewer.process(draft)

        return {
            "status": "success",
            "final_draft": draft,
            "review": review,
            "examples": relevant_examples,
            "justification": example_justification,
        }


def process_emails(emails_list):
    email_system = EmailProcessingSystem(client)
    results = {}

    for email in emails_list:
        result = email_system.process_email(email)
        results[email] = result

    return results


# Add this before create_gradio_interface()
sample_emails = [
    """Hi, I need to get my medical records from last month's visit.
    Can you help me with the process? Thanks!""",
    """Hello, I'm trying to schedule an appointment for next week.
    I have Aetna insurance and wanted to confirm you accept it before booking.""",
    """My appointment is tomorrow at 2pm but something urgent came up at work.
    Is there any way I could reschedule?""",
    """Running low on my blood pressure medication.
    Need to get it refilled before next week. What's the process?""",
    """Hi there, just moved to the area and looking for a new primary care doctor.
    Are you accepting new patients with United Healthcare?""",
]


def create_gradio_interface():
    state = {
        "current_index": 0,
        "approved_count": 0,
        "disapproved_count": 0,
        "results": {},
    }

    email_system = EmailProcessingSystem(client)

    def process_current_email(email_index):
        email = sample_emails[email_index]
        result = email_system.process_email(email)
        state["results"][email] = result

        return (
            result["final_draft"],
            result["examples"],
            result["justification"],
            state["approved_count"],
            state["disapproved_count"],
        )

    def update_email_display(index):
        return sample_emails[index]

    def approve_response():
        state["approved_count"] += 1
        return {
            approve_btn: gr.Button(interactive=False),
            disapprove_btn: gr.Button(interactive=False),
            approved_count: state["approved_count"],
        }

    def disapprove_response():
        state["disapproved_count"] += 1
        return {
            approve_btn: gr.Button(interactive=False),
            disapprove_btn: gr.Button(interactive=False),
            disapproved_count: state["disapproved_count"],
        }

    # Create the Gradio interface
    with gr.Blocks(title="Medical Email Processing System") as interface:
        gr.Markdown("# 🏥 Medical Email Processing System")
        gr.Markdown("### Process and review medical email responses with AI assistance")

        with gr.Row():
            email_index = gr.Slider(
                minimum=0,
                maximum=len(sample_emails) - 1,
                step=1,
                value=0,
                label="📧 Email Number",
                interactive=True,
            )

            with gr.Column():
                approved_count = gr.Number(
                    value=0, label="✅ Approved Emails", interactive=False
                )
                disapproved_count = gr.Number(
                    value=0, label="❌ Disapproved Emails", interactive=False
                )

        original_email = gr.Textbox(
            label="📥 Original Email",
            lines=4,
            scale=2,
            container=True,
        )

        with gr.Row():
            process_btn = gr.Button("🔄 Process This Email", variant="primary")
            skip_btn = gr.Button("⏭️ Skip This Email", variant="secondary")

        # Output displays
        draft_response = gr.Textbox(
            label="📝 Draft Response",
            lines=4,
            container=True,
        )
        examples = gr.Textbox(
            label="📚 Similar Examples",
            lines=8,
            container=True,
        )
        justification = gr.Textbox(
            label="💡 Example Justification",
            lines=2,
            container=True,
        )

        with gr.Row():
            approve_btn = gr.Button(
                "✅ Approve Draft", interactive=False, variant="success"
            )
            disapprove_btn = gr.Button(
                "❌ Disapprove Draft", interactive=False, variant="stop"
            )

        # Event handlers
        email_index.change(
            fn=update_email_display,
            inputs=[email_index],
            outputs=[original_email],
        )

        approve_btn.click(
            fn=approve_response, outputs=[approve_btn, disapprove_btn, approved_count]
        )

        disapprove_btn.click(
            fn=disapprove_response,
            outputs=[approve_btn, disapprove_btn, disapproved_count],
        )

        process_btn.click(
            fn=process_current_email,
            inputs=[email_index],
            outputs=[
                draft_response,
                examples,
                justification,
                approved_count,
                disapproved_count,
            ],
        ).then(
            lambda: {
                approve_btn: gr.Button(interactive=True),
                disapprove_btn: gr.Button(interactive=True),
            },
            outputs=[approve_btn, disapprove_btn],
            api_name=False,
        )

        skip_btn.click(
            fn=process_current_email,
            inputs=[email_index],
            outputs=[
                draft_response,
                examples,
                justification,
                approved_count,
                disapproved_count,
            ],
        )

        # Initialize the original email display with the first email
        original_email.value = sample_emails[0]

    return interface


if __name__ == "__main__":
    interface = create_gradio_interface()
    interface.launch()


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


## **Task 1: Create your own Email Processing System for your own domain!**

Modify the code above **Module 7: AI Email Processing with Gradio User Interface** to create your own specialized Email Processing System with different external knowledge, text response.

You can specialize your system for various use cases (preferably for your own personal use case), such as:
- ✅ Legal Assistant – Automate legal inquiries and document processing.
- ✅ Customer Support AI – Provide automated replies and categorize customer issues.
- ✅ Healthcare Email Assistant – Process appointment requests and medical inquiries.
- ✅ Marketing Automation – Categorize leads, analyze engagement, and send follow-ups.

In [38]:
# 🔹 Import Required Libraries
import gradio as gr
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class GitHubEmailResponseRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2", use_auth_token=HGToken)
        
        # Sample GitHub-related email responses
        self.examples = {
            "repo_access": """
                ORIGINAL EMAIL:
                Hi, I need access to the private GitHub repo for our project.

                MY RESPONSE:
                Hi there! To gain access, please provide your GitHub username.
                We'll review your request and grant the appropriate permissions within 24 hours.
                Let me know if you need anything else!
            """,
            "merge_conflict": """
                ORIGINAL EMAIL:
                I'm having trouble merging my pull request due to conflicts. Any help?

                MY RESPONSE:
                Hey! Merge conflicts happen when changes overlap. Here’s how to fix them:
                1. Fetch the latest updates: `git pull origin main`
                2. Resolve conflicts manually
                3. Commit and push changes
                If you need guidance, I can walk you through it!
            """,
            "branch_protection": """
                ORIGINAL EMAIL:
                Why can’t I push directly to the main branch?

                MY RESPONSE:
                Hello! Our repo has **branch protection** enabled to prevent direct pushes to `main`.
                You’ll need to open a **pull request**, and once approved, it can be merged safely.
                Let me know if you need help submitting one!
            """,
            "repo_deletion": """
                ORIGINAL EMAIL:
                How do I delete a GitHub repository?

                MY RESPONSE:
                Hi! To delete a repo:
                1. Go to **Settings** in your repo.
                2. Scroll to the bottom and click **Delete this repository**.
                3. Confirm by typing the repo name.
                Be careful—this action is irreversible!
            """,
        }

        # Pre-compute embeddings for GitHub responses
        self.example_embeddings = {
            k: self.encoder.encode(v) for k, v in self.examples.items()
        }

    def get_relevant_response(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.example_embeddings.items()
        }
        sorted_examples = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_examples = []
        for example_name, score in sorted_examples[:top_k]:
            if score > 0.3:
                relevant_examples.append(self.examples[example_name])

        return "\n\n".join(relevant_examples) if relevant_examples else "No relevant example found."


class GitHubEmailAgent:
    def __init__(self, client):
        self.client = client
        self.response_retriever = GitHubEmailResponseRetriever()

    def process_email(self, email_content):
        # Get relevant examples for the email content
        relevant_examples = self.response_retriever.get_relevant_response(email_content)

        # Generate AI Draft Response
        draft_response = f"""SYSTEM: Suggested GitHub Support Response

        📌 **Issue Detected**:
        {email_content}

        📝 **Suggested Response**:
        {relevant_examples}
        """

        return draft_response, relevant_examples


class GitHubEmailProcessingSystem:
    def __init__(self, client):
        self.agent = GitHubEmailAgent(client)

    def process_email(self, email_content):
        draft, examples = self.agent.process_email(email_content)
        return {
            "final_draft": draft,
            "examples": examples,
        }


# 📧 Sample GitHub Support Emails
sample_github_emails = [
    "Hello, I can't push to the main branch. Is something wrong?",
    "Hi, I need help resolving a merge conflict in my pull request.",
    "How do I transfer ownership of my GitHub repository?",
    "Can I delete a repository permanently?",
    "What's the best way to grant access to a private repo?",
]


# 🔹 Create Gradio UI for Email Processing
def create_gradio_interface():
    state = {
        "current_index": 0,
        "approved_count": 0,
        "disapproved_count": 0,
        "results": {},
    }

    email_system = GitHubEmailProcessingSystem(client=None)

    def process_current_email(email_index):
        email = sample_github_emails[email_index]
        result = email_system.process_email(email)
        state["results"][email] = result

        return (
            result["final_draft"],
            result["examples"],
            state["approved_count"],
            state["disapproved_count"],
        )

    def update_email_display(index):
        return sample_github_emails[index]

    def approve_response():
        state["approved_count"] += 1
        return {
            approve_btn: gr.Button(interactive=False),
            disapprove_btn: gr.Button(interactive=False),
            approved_count: state["approved_count"],
        }

    def disapprove_response():
        state["disapproved_count"] += 1
        return {
            approve_btn: gr.Button(interactive=False),
            disapprove_btn: gr.Button(interactive=False),
            disapproved_count: state["disapproved_count"],
        }

    # 🎨 Build Gradio Interface
    with gr.Blocks(title="GitHub Support Email Assistant") as interface:
        gr.Markdown("# 🏆 GitHub Email Support System")
        gr.Markdown("### Process GitHub-related support emails with AI assistance")

        with gr.Row():
            email_index = gr.Slider(
                minimum=0,
                maximum=len(sample_github_emails) - 1,
                step=1,
                value=0,
                label="📧 Email Number",
                interactive=True,
            )

            with gr.Column():
                approved_count = gr.Number(
                    value=0, label="✅ Approved Answer", interactive=False
                )
                disapproved_count = gr.Number(
                    value=0, label="❌ Disapproved Answer", interactive=False
                )

        original_email = gr.Textbox(
            label="📥 Original Question",
            lines=4,
            scale=2,
            container=True,
        )

        with gr.Row():
            process_btn = gr.Button("🔄 Process This Question", variant="primary")
            skip_btn = gr.Button("⏭️ Skip This Question", variant="secondary")

        # Output displays
        draft_response = gr.Textbox(
            label="📝 AI Draft Response",
            lines=4,
            container=True,
        )
        examples = gr.Textbox(
            label="📚 Similar Examples",
            lines=8,
            container=True,
        )

        with gr.Row():
            approve_btn = gr.Button("✅ Approve Draft", interactive=False, variant="success")
            disapprove_btn = gr.Button("❌ Disapprove Draft", interactive=False, variant="stop")

        # 🔹 Event Handlers
        email_index.change(
            fn=update_email_display,
            inputs=[email_index],
            outputs=[original_email],
        )

        approve_btn.click(
            fn=approve_response, outputs=[approve_btn, disapprove_btn, approved_count]
        )

        disapprove_btn.click(
            fn=disapprove_response, outputs=[approve_btn, disapprove_btn, disapproved_count]
        )

        process_btn.click(
            fn=process_current_email,
            inputs=[email_index],
            outputs=[draft_response, examples, approved_count, disapproved_count],
        ).then(
            lambda: {approve_btn: gr.Button(interactive=True), disapprove_btn: gr.Button(interactive=True)},
            outputs=[approve_btn, disapprove_btn],
            api_name=False,
        )

        skip_btn.click(
            fn=process_current_email,
            inputs=[email_index],
            outputs=[draft_response, examples, approved_count, disapproved_count],
        )

        # Initialize the original email display
        original_email.value = sample_github_emails[0]

    return interface


# 🚀 Launch Gradio UI
if __name__ == "__main__":
    interface = create_gradio_interface()
    interface.launch()


* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.



## Challenge
Create a video of your Email Procesing System running for 10-60 seconds and share it as a demowith the group.

## 🎉 Congratulations! 🎉
You've successfully created yet another AI-powered Python application with a user-friendly interface!
